In [1]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib as mpl
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

import scipy

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
import cage_utils
plt.rcParams['figure.figsize'] = [14, 10]

In [2]:
def asymTrapFilter(wf_in, rise, flat, fall):
    """ Computes an asymmetric trapezoidal filter"""
    wf_out = np.zeros(len(wf_in))
    wf_out[0] = wf_in[0]/float(rise)
    for i in range(1, rise):
        wf_out[i] = wf_out[i-1] + (wf_in[i])/float(rise)
    for i in range(rise, rise+flat):
        wf_out[i] = wf_out[i-1] + (wf_in[i] - wf_in[i-rise])/float(rise)
    for i in range(rise+flat, rise+flat+fall):
        wf_out[i] = wf_out[i-1] + (wf_in[i] - wf_in[i-rise])/float(rise) - wf_in[i-rise-flat]/float(fall)
    for i in range(rise+flat+fall, len(wf_in)):
        wf_out[i] = wf_out[i-1] + (wf_in[i] - wf_in[i-rise])/float(rise) - (wf_in[i-rise-flat] - wf_in[i-rise-flat-fall])/float(fall)
        
    return(wf_out)

In [3]:
def time_point_thresh(wf_in, threshold, tp_max):
    """
    Find the last timepoint before tp_max that wf_in crosses a threshold
     wf_in: input waveform
     threshold: threshold to search for
     tp_out: final time that waveform is less than threshold
    """
    for i in range(tp_max, 0, -1):
        if(wf_in[i]>threshold and wf_in[i-1]<threshold):
            tp_out = i
    return tp_out

In [4]:
f_superpulse = './data/normScan/run64_superpulses.hdf5'

f_pulser = './data/superpulses_oct2020.h5'
# f_pulser = './data/CAGE_amp_long_transient_response.csv'

In [5]:
# with pd.HDFStore(f_pulser, 'r') as f:
#     print("Keys:", f.keys())
with pd.HDFStore(f_superpulse, 'r') as f:
    print("Keys:", f.keys())
    
data_superpulse = pd.read_hdf(f_superpulse, key = '/superpulses')
data_pulser = pd.read_hdf(f_pulser, key = '/superpulses') 

Keys: ['/superpulses']


In [14]:
print(data_pulser.head())

   V_pulser  run  E_keV  mV_firststage  index           unique_key  YYYY  mm  \
0      0.16  838     60           13.2    836  cage-cyc838-2020107  2020  10   

   dd  cycle  ...                   dsp_file dsp_path  \
0   7    838  ...  cage_run58_cyc838_dsp.lh5     /dsp   

                    hit_file hit_path     startTime  threshold    daq_gb  \
0  cage_run58_cyc838_hit.lh5     /hit  1.602109e+09       16.0  1.556451   

       stopTime   runtime                                         superpulse  
0  1.602110e+09  9.997692  [-0.00038934359960989353, 0.005904600270233672...  

[1 rows x 28 columns]


In [ ]:
cols = ['time', 'v_out1', 'v_out']
data_pulser=pd.read_csv(f_pulser, delimiter=' ,', header=0, names = cols)
raw_impulse = np.array(data_pulser['v_out'])
raw_time = np.array(data_pulser['time'])
print(len(raw_impulse))

In [ ]:
print(raw_time)

In [ ]:
time_full = raw_time[-1]-raw_time[0]
time_delta = time_full/len(raw_time)
print(time_delta)
10e-9/time_delta

In [ ]:
new_time = raw_time[300::4]
sampled_impulse = raw_impulse[300::4]
print(len(new_time))
print(sampled_impulse[0])

In [ ]:
plt.plot(sampled_impulse)
plt.xlim(900, 950)

In [ ]:
bl = np.mean(new_impulse[:800])
print(bl)

In [ ]:
impulse_blSub = sampled_impulse - bl
impulse_norm = impulse_blSub/np.amax(impulse_blSub)

In [ ]:
plt.plot(impulse_norm)

In [6]:
raw_pulser_super = data_pulser['superpulse'][0]
pulser_super_notched = cage_utils.notchFilter_SIS3302(raw_pulser_super, Q=20)
pulser_bl_mean = np.mean(pulser_super_notched[100:3500])
pulser_super = pulser_super_notched - pulser_bl_mean

# pulser_super = impulse_norm

raw_superpulse_60 = data_superpulse['pure_60'][0]
raw_superpulse_60_notched = cage_utils.notchFilter_SIS3302(raw_superpulse_60, Q=20)
bl_mean_60 = np.mean(raw_superpulse_60_notched[100:3500])
print(bl_mean_60)
superpulse_60 = raw_superpulse_60_notched - bl_mean_60

0.00034926446361611925


In [10]:
times = data_superpulse['samples'][0]

In [15]:
print(len(pulser_super))

8192


In [13]:
wf_dict = {'WF type': 'superpulse_60keV', 'samples': [times], 'waveform': [superpulse_60]}
wf_df = pd.DataFrame(data=wf_dict)
print(wf_df)

            WF type                                            samples  \
0  superpulse_60keV  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                            waveform  
0  [-0.0013797931269024445, 0.002320059595536762,...  


In [17]:
wf_dict2 = {'WF type': 'pulser_superpulse', 'samples': [times], 'waveform': [pulser_super]}
wf_df_temp = pd.DataFrame(data=wf_dict2)
print(wf_df_temp)

             WF type                                            samples  \
0  pulser_superpulse  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                            waveform  
0  [-0.0019576910764730025, 0.00140125363761926, ...  


In [18]:
wf_df = wf_df.append(wf_df_temp)

In [19]:
outfile = f'./data/normScan/superpulses_forRecyo.hdf5'
wf_df.to_hdf(outfile, key='superpulses', mode='w')

/global/homes/g/gothman/.local/lib/python3.8/site-packages/pandas/core/generic.py:2603: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['WF type', 'samples', 'waveform'], dtype='object')]

  pytables.to_hdf(


In [ ]:
test = cage_utils.notchFilter(raw_superpulse_60, 25e6, 20)
test2 = cage_utils.notchFilter(test, 50e6, 20)
print(len(test))
print(len(test2))

In [ ]:
print(len(superpulse_60))
print(data_superpulse['samples'][0])

In [ ]:
plt.plot(pulser_super)
# plt.ylim(-0.025, 0.25)

In [ ]:
times = raw_superpulse_60 = data_superpulse['samples'][0]
print(len(times))
print(len(raw_superpulse_60_notched))

In [ ]:
plt.plot(times, superpulse_60[1:])
print(len(superpulse_60))

In [ ]:
trap_super60 = asymTrapFilter(superpulse_60, 2, 100, 400)
max_trap_60 = np.argmax(trap_super60)
tp_0_60 = time_point_thresh(trap_super60, 0.002, max_trap_60)
print(tp_0_60)
# print(trap_super60[3955])

In [ ]:
plt.plot(superpulse_60[:-1] )
plt.plot(trap_super60[:-1])
plt.title('physics superpulse')
plt.axvline(tp_0_60)
# plt.xlim(3800, 4200)
plt.xlim(3950, 3960)
# plt.xlim(0, 3800)
plt.ylim(-0.01, 0.2)

In [ ]:
trap_pulser = asymTrapFilter(pulser_super, 2, 100, 400)
max_trap_pulser = np.argmax(trap_pulser)
tp_0_pulser = time_point_thresh(trap_pulser, 0.002, max_trap_pulser)
print(tp_0_pulser)
print(pulser_super[tp_0_pulser])

In [ ]:
plt.plot(pulser_super[:-1])
plt.plot(trap_pulser[:-1])
plt.axvline(tp_0_pulser)
# plt.axvline(919)
# plt.xlim(3800, 4200)
plt.ylim(-0.001, 0.005)
plt.xlim(910, 930)
plt.title('pulser superpulse')

In [ ]:
# # # pulser_super_notched_zeroed = [0.0 for ind in enumerate(pulser_super_notched) if int(ind)<tp_0_pulser]
# # pulser_super_notched_zeroed = [0.0 for i in range(len(pulser_super_notched)) if i< tp_0_pulser]
# # print(pulser_super_notched_zeroed)
# pulser_super_zeroed = pulser_super

# for i in range(len(pulser_super), 20000):
#     if i < tp_0_pulser:
#         pulser_super_zeroed[i] = 0.0
        
# print(pulser_super_zeroed)

# # (index, element) for index, element in enumerate(a_list)

In [ ]:
delta = tp_0_60 - tp_0_pulser
print(delta)

from scipy.ndimage.interpolation import shift

shift_pulser_super = shift(pulser_super, delta, cval=0)
print(shift_pulser_super )

In [ ]:
plt.plot(shift_pulser_super[:8191], '-r')
plt.plot(superpulse_60[:-1])
plt.xlim(3800, 4200)

In [ ]:
shift_pulser_super[tp_0_60-1]

In [ ]:
impulse_1 = shift_pulser_super[tp_0_60-1:8191]

In [ ]:
plt.plot(impulse_1)
plt.plot(superpulse_60)

In [ ]:
impulse = impulse_1[::10]

In [ ]:
plt.plot(impulse)
plt.plot(superpulse_60[:8000])

In [ ]:
fft_impulse = scipy.fft.fft(impulse)#, norm='forward'
fft_60 = scipy.fft.fft(superpulse_60[:8000]) 

In [ ]:
hmm = scipy.signal.deconvolve(fft_60, fft_impulse)

In [ ]:
hmm_deconvo = scipy.fft.ifft(hmm[0])
hmm_1 = scipy.fft.ifft(hmm[1])

In [ ]:
plt.plot(hmm_deconvo)
# plt.xlim(1000, 6000)
# plt.ylim(0.0002, 0.0012)

In [ ]:
plt.plot(hmm_deconvo)
plt.xlim(1000, 6000)
# plt.ylim(0.0002, 0.0012)

In [ ]:
plt.plot(hmm_1)

In [ ]:
deconvo = scipy.signal.deconvolve(superpulse_60[:8000], impulse)

In [ ]:
print(len(deconvo[0]))

In [ ]:
print(deconvo[0])

In [ ]:
plt.plot(deconvo[0])

In [ ]:
fft_impulse = scipy.fft.fft(impulse)#, norm='forward'
fft_60 = scipy.fft.fft(superpulse_60[:8000]) 

In [ ]:
plt.plot(fft_impulse)
plt.title('FFT of pulser superpulse')

In [ ]:
plt.plot(fft_60[:-1])
plt.title('FFT of physics superpulse')

In [ ]:
deconvo_fft = np.divide(fft_60, fft_impulse)
deconvo = scipy.fft.ifft(deconvo_fft)

In [ ]:
plt.plot(deconvo_fft)
plt.title('FFT_physics/FFT_pulser')

In [ ]:
plt.plot(deconvo)
plt.title('IFFT of FFT_physics/FFT_pulser')

In [ ]:
plt.plot(aligned_pulser_super_zeroed )
plt.xlim(3950, 3960)

In [ ]:
aligned_pulser_super_zeroed_snipped = aligned_pulser_super_zeroed[:8192]
pulser_super_snipped = aligned_pulser_super_zeroed[:8192]
# impulse = aligned_pulser_super_zeroed_snipped
impulse_1 = pulser_super_snipped

In [ ]:
plt.plot(impulse_1, '-r')
plt.plot(superpulse_60 )
plt.xlim(3900, 4000)

In [ ]:
impulse = impulse_1[tp_0_pulser+1::5]

In [ ]:
plt.plot(impulse)

In [ ]:
impulse_1[3957]

In [ ]:
hmm = scipy.signal.deconvolve(fft_60, fft_impulse)

In [ ]:
print(hmm[1])

In [ ]:
hmm_deconvo = scipy.fft.ifft(hmm[1])

In [ ]:
plt.plot(hmm_deconvo)
# plt.xlim(3950, 3970)

In [ ]:
plt.plot(hmm[1]*-1)

In [ ]:
print(len(pulser_super))

In [ ]:
run = 60 #44 #70 #64
dsp_id = '02'
user = True
hit =True
cal = True
lowE=False
etype = 'trapEftp_cal'

corr_DCR=True
corr_AoE=True
corr_ToE=True

norm = True
cut = True


dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)

df = cage_utils.apply_DC_Cuts(run, df_raw)

df_60 = df.query('trapEftp_cal > 55 and trapEftp_cal < 65').copy()


if corr_ToE==True:
    ToE_mode = cage_utils.mode_hist(df, param='ToE', a_bins=1000, alo=-0.1, ahi=0.45, cut=False, cut_str='')
    df['ToE_plot'] = df['ToE'] - ToE_mode
    df_60['ToE_plot'] = df_60['ToE'] - ToE_mode

In [ ]:
t, window_1_wfs = cage_utils.get_wfs(df_60, dg, cut_bkg_low, all=True)

In [ ]:
# fig, ax = plt.subplots(figsize=(9,8))
ax = plt.axes()

# set up colorbar to plot waveforms of different energies different colors
colors = plt.cm.viridis(np.linspace(0, 1, len(window_1_wfs)))
c = np.arange(0, len(window_1_wfs))
norm = mpl.colors.Normalize(vmin=c.min(), vmax=c.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
cmap.set_array([])

for n in range(len(window_1_wfs)):
    plt.plot(t, window_1_wfs[n][:len(window_1_wfs[n])-1], c=cmap.to_rgba(n))

# cb = fig.colorbar(cmap, ticks=(57.5, 59.5))
cb.set_label("Energy", ha = 'right', va='center', rotation=270, fontsize=20)
cb.ax.tick_params(labelsize=18)

ax.text(0.95, 0.83, f'{cut_bkg_hi}', verticalalignment='top',
                        horizontalalignment='right', transform=ax.transAxes, color='green', fontsize=14,
                        bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 10})

plt.title(f'run {run}: r = {radius} mm \ntheta = {angle_det} deg')

#     plt.xlim(3800, 8000)
#     plt.ylim(0.4, 1.01)
plt.setp(ax.get_xticklabels(), fontsize=16)
plt.setp(ax.get_yticklabels(), fontsize=16)
plt.title(f'60 keV waveforms for run {run}\nr = {radius} mm; theta = {angle_det} deg', fontsize=20)
plt.xlabel('clock cycles', fontsize=20)
# plt.xlim(3900, 4050)